In [1]:
"""Polarizable continuum model (PCM) solvent model."""

__author__    = "Gabriel Gerez"
__credit__    = ["Gabriel Gerez"]

__date__      = "2023-08-24"

In [2]:
from vampyr import vampyr3d as vp
import numpy as np
from scipy.special import erf

# Polarizable continuum model (PCM) for solvation

PCM treats the solvent as an isotropic continuum characterized by its permittivity, obtained from the 
solvent's bulk properties. The solute is described at the quantum level, and their interaction is 
represented through an electrostatic potential supported on the so-called cavity of the solute. Below, we 
detail the mathematical formulation of this model and provide a practical example of how it can be 
implemented.

The cavity is the area where the solute is located and we define it using an interlocking spheres 
model where the spheres are centered in the atoms of the solute and the radii of the spheres is usually the 
vdW-radii.

In our case we define a cavity function as 
$$
C(\vec{r}) = \begin{cases}
    0 & \mathbf{r}\text{ ~inside cavity} \\
    1 & \mathbf{r} \text{ ~outside cavity}
\end{cases}
$$
We want the boundary between the inside and outside of the cavity to be smooth. 


Each i-th sphere is defined by a center $\mathbf{r}_i$ and a radius $R_i$ and the cavity function is defined as
$$
C_i(\mathbf{r}) = \frac{1}{2} \left( 1 + \text{erf} \left( \frac{|\mathbf{r} - \mathbf{r}_i| - R_i}{ \sigma} \right) \right),
$$
where erf is the error function and $\sigma$ is a parameter that controls the width of the boundary.

The total cavity function is then defined as
$$
C(\mathbf{r}) = 1 - \prod_i \left(1-C_i(\mathbf{r})\right).
$$

This has been implemented as a class below

In [3]:
class Cavity(object):
    
    def __init__(self, cav_coords, radii, width):
        self.r_list = cav_coords # list of cavity centers. Normally, but not always, nucleus coordinates. 
        self.R_list = radii  # list of cavity radii. Normally, but not always, nucleus radii.
        self.sigma = width  # width of the cavity boundary
   

    def __call__(self,  r):
        """
        r is a list of floats of length 3, can be a numpy array as well
        """
        r_vec = np.array(r)
        C = 1.0
        for i, r_i in enumerate(self.r_list):
            r_vec_i = np.array(r_i)
            s_i = np.linalg.norm(r_vec_i - r_vec) - self.R_list[i]
            O_i = (1.0/2.0)*(1 + erf(s_i/self.sigma))
            C_i = 1 - O_i
            C *= 1 - C_i
        C = 1.0 - C
        return C



The electrostatic potential $V(\mathbf{r})$ satisfies the generalized Poisson equation (GPE)
$$
    \nabla \cdot (\varepsilon(\mathbf{r}) \nabla V(\mathbf{r})) = -4\pi \rho(\mathbf{r}),
$$
where $\rho(\mathbf{r}$ is the total total density of the solute including both electronic, $\rho_{el}$ and nuclear $\rho_{nuc}$ densities,
$$
    \rho(\mathbf{r}) = \rho_{el}(\mathbf{r}) +  \rho_{nuc}(\mathbf{r}),
$$
and $\varepsilon(\mathbf{r})$ is the position dependent permittivity parametrized using the cavity of the solute where
$$
    \varepsilon(\mathbf{r}) = \begin{cases}
        \varepsilon_0 & \mathbf{r}~\text{inside}~\text{cavity} \\
        \varepsilon_r & \mathbf{r}~\text{outside}~ \text{cavity}.
    \end{cases}
$$
where $\varepsilon_0$ is the permittivity of free space and $\varepsilon_r$ is the relative permittivity of the solvent.


We have defined the permittivity function as follows
$$
\varepsilon(\mathbf{r}) = \varepsilon_0  \exp\left(\log\left[\frac{\varepsilon_r}{ \varepsilon_0}\right]\left(1 - C(\mathbf{r})\right)\right).
$$

This has been implemented as a class below

In [4]:
class Permittivity():
    def __init__(self, cavity, inside=1.0, outside=2.0):
        
        self.C = cavity # instance of cavity class
        self.inside = inside # permittivity of free space
        self.outside = outside # permittivity of solvent
    
    def __call__(self, r):
        C_eval = self.C(r)
        permittivity = self.inside*np.exp((np.log((self.outside/self.inside)))*(1.0 - C_eval))
    
        return permittivity



The electrostatic potential can be divided into two contributions, the solute-solvent interaction reaction potential $V_R$ (also normally simply called reaction potential), and the vacuum potential $V_{vac}$,
$$
    V(\mathbf{r}) = V_R(\mathbf{r}) + V_{vac}(\mathbf{r}),
$$
where 
$$
    \nabla^2 V_{vac}(\mathbf{r}) = -4\pi \rho(\mathbf{r}).
$$

The energy contribution from the reaction potential, $E_R$ is given as
$$
    E_R =\frac{1}{2} \int d\mathbf{r}V_R(\mathbf{r})\rho(\mathbf{r}).
$$
Since this energy is only dependent on the reaction potential, we want to directly solve for it.


Resolving the chain rule and manipulating the equation gives us the following expression for the reaction potential
$$
     \nabla^2  V_R(\mathbf{r})  = -4\pi\left( \frac{\rho(\mathbf{r})}{\varepsilon(\mathbf{r})} -\rho(\mathbf{r})\right) - \frac{\nabla \varepsilon(\mathbf{r}) \cdot \nabla V(\mathbf{r})}{\varepsilon(\mathbf{r})}
$$
We solve the equation by using the Poisson operator $P(\mathbf{r})$ to directly solve for the 
reaction potential
$$
    V_R(\mathbf{r})  = P(\mathbf{r}) \star\left[ \rho_{eff}(\mathbf{r}) + \gamma_s(\mathbf{r}) \right]
$$
where we did the following substitutions
$$
    \rho_{eff}(\mathbf{r}) = 4\pi \left(\frac{\rho(\mathbf{r})}{\varepsilon(\mathbf{r})} - \rho(\mathbf{r})\right) \\ 
    ~\\
    \gamma_s(\mathbf{r}) = \frac{\nabla \varepsilon(\mathbf{r}) \cdot \nabla V(\mathbf{r})}{\varepsilon(\mathbf{r})}
$$
which we call respectively the effective molecular density $\rho_{eff}$ and the surface charge distribution $\gamma_s$.

Note that the Reaction potential must be solved iteratively since it appears on both sides of the last equation through the total electrostatic potential $V$ which is part of $\gamma_s$ as shown in the equations above.

Below is an implementation of the self consistent procedure for the reaction potential for a single positive charge inside a sphere cavity of unit radius.
We start by defining several functions to compute the objects needed for the reaction potential

In [5]:
def constructChargeDensity(positions, charges, width_parameter):
    """Computes the charge density of a set of point charges using a Gaussian
    function. The Gaussian function is centered at the position of the charge
    and the width parameter is the standard deviation of the Gaussian.

    Args:
        positions (list of lists of floats): list of positions of the charges
        charges (list of floats): list of charges
        width_parameter (int, optional): Standard deviation of the Gaussian. Defaults to 1000.

    Returns:
        function: function that computes the charge density at a given position
    """
    charge_density = vp.GaussExp()
    for (pos, charge) in zip(positions, charges):
        beta = width_parameter
        alpha = (beta / np.pi)**(3.0/2.0)
        charge_density.append(vp.GaussFunc(beta=beta, alpha=alpha*charge, position=pos, poly_exponent=[0,0,0]))
    return charge_density


def computeGamma(Derivative_op, V, permittivity, epsilon):
    """Computes gamma_s for a given permittivity and potential.

    Args:
        Derivative_op (vp.ABGVDerivative): Derivative operator
        V (vp.FunctionTree): Potential used to compute gamma_s
        permittivity (vp.FunctionTree): permittivity function of the solvent
        epsilon (float): crop precision

    Returns:
        _type_: _description_
    """
    gamma =  vp.dot(vp.gradient(Derivative_op,V), vp.gradient(Derivative_op, permittivity)) * ( permittivity**(-1))
    gamma = gamma.crop(epsilon)
    return gamma.deepCopy()



Initialize the Multiresolution analysis

In [6]:
k = 5
L = [-10, 10]
MRA = vp.MultiResolutionAnalysis(order=k, box=L)
print(MRA)

                    MultiResolution Analysis                    
----------------------------------------------------------------
 polynomial order      : 5
 polynomial type       : Interpolating
----------------------------------------------------------------
 total boxes           : 8
 boxes                 : [          2           2           2 ]
 unit lengths          : [  10.000000   10.000000   10.000000 ]
 scaling factor        : [   1.250000    1.250000    1.250000 ]
 lower bounds          : [ -10.000000  -10.000000  -10.000000 ]
 upper bounds          : [  10.000000   10.000000   10.000000 ]
 total length          : [  20.000000   20.000000   20.000000 ]



Initialize all operators that will be used

In [7]:
epsilon = 1e-5
Projection_op = vp.ScalingProjector(mra=MRA, prec=epsilon)
Derivative_op = vp.ABGVDerivative(mra=MRA, a=0.0, b=0.0)
Poisson_op = vp.PoissonOperator(mra=MRA, prec=epsilon)

Compute the density of the solute

In [8]:
charge_coords = [[0.0000000000,    0.0000000000,    0.000000000]]
charges = [1.0]
charge_width = 1000.0
dens = Projection_op(constructChargeDensity(charge_coords, charges, width_parameter=charge_width))


Project the permittivity of the solvent

In [9]:
cav_coords = [[0.0000000000,    0.0000000000,    0.000000000]] # list of cavity centers. Normally, but not always, nucleus coordinates.
cav_radii = [1.0] # list of cavity radii. Normally, but not always, nucleus radii.
boundary_width = 0.2 # width of the cavity boundary

C = Cavity(cav_coords, cav_radii, boundary_width)
perm = Projection_op(Permittivity(C, inside=1.0, outside=2.0))

Construct the effective solute density as defined above and compute the vacuum potential $V_{vac}$

In [10]:
rho_eff = (4*np.pi)*((dens * (perm)**(-1))  - (dens))
V_vac =Poisson_op((4*np.pi)*(dens))

Compute the zero-th reaction potential before the iterative procedure

In [11]:
gamma_0 = computeGamma(Derivative_op, V_vac, perm, epsilon)
V_R = Poisson_op(rho_eff + gamma_0)

Iterate throught the application of the Poisson operator to compute the reaction potential

In [14]:
update = 1.0
E_r_old = 0.0
max_iter = 100
for i in range(max_iter):
    V_tot = V_vac + V_R
                
    # compute the surface charge distribution 
    gamma_s = computeGamma(Derivative_op, V_tot, perm, epsilon)

    # solve the generalized poisson equation for V_R 
    V_R_np1 = Poisson_op((rho_eff) + (gamma_s)) 
    dV_R = V_R_np1 - V_R
    
    update = dV_R.norm()
    
    V_R =  V_R + dV_R
    
    E_r = (1/2)*vp.dot(V_R, dens) # computing energy
    
    dE_r = E_r - E_r_old
    E_r_old = E_r
    
    print(f"iteration: {i} \t norm: {V_R.norm()} \t update: {update} \t energy: {E_r} \t energy update: {dE_r}") 
    
    if (update < epsilon):
        break
else:
    print("WARNING: SCRF did not converge")

iteration: 0 	 norm: 6.025407139395362 	 update: 8.710992846061201e-08 	 energy: -0.2626682445748242 	 energy update: -0.2626682445748242


The energy can then be computed as follows

In [ ]:
E_r = (1/2)*vp.dot(V_R, dens)